<a href="https://colab.research.google.com/github/toche7/DataAnalytic/blob/main/Lab01EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# การตรวจสอบและจัดการ Data Errors และ Missing Values เบื้องต้น

## 1. Import Libraries และโหลดข้อมูล

ในส่วนนี้ เราจะนำเข้าไลบรารีที่จำเป็นและโหลดข้อมูลตัวอย่างเพื่อใช้ในการวิเคราะห์

In [ ]:
# prompt: import titanic data

!wget https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv

import pandas as pd

# Load the dataset
df = pd.read_csv('titanic.csv')

# Display the first few rows to understand the data
print(df.head())

In [ ]:
df.info()

In [ ]:
import pandas as pd
import missingno as msno

# โหลด DataFrame ตัวอย่าง (สามารถแทนที่ด้วยไฟล์ข้อมูลของคุณ)
# ตัวอย่าง: สร้าง DataFrame สุ่มเพื่อสาธิต
import numpy as np
data = {
    'col1': np.random.rand(10),
    'col2': np.random.randint(1, 100, 10),
    'col3': ['A', 'B', 'C', 'A', 'D', 'B', 'C', 'A', 'D', 'E']
}
df = pd.DataFrame(data)

# เพิ่มค่า NaN เพื่อสาธิต missing values
df.loc[[2, 5], 'col1'] = np.nan
df.loc[7, 'col2'] = np.nan
df.loc[0, 'col3'] = np.nan

print("แสดง DataFrame ตัวอย่าง:")
display(df)

## 2. ตรวจสอบ Data Errors เบื้องต้น

เราจะทำการตรวจสอบประเภทข้อมูลของแต่ละคอลัมน์และแปลงค่าที่ไม่ถูกต้องให้เป็นตัวเลข

### 2.1 ตรวจสอบ dtype ของแต่ละคอลัมน์

In [ ]:
print("\nตรวจสอบ dtype ของแต่ละคอลัมน์:")
display(df.dtypes)

### 2.2 แปลงค่าที่ผิดรูปเป็นตัวเลขด้วย `pd.to_numeric(errors='coerce')`

คำสั่งนี้จะพยายามแปลงค่าในคอลัมน์ให้เป็นตัวเลข หากแปลงไม่ได้ จะแทนที่ด้วย `NaN` (Not a Number)

In [ ]:
# สร้างคอลัมน์ที่มีค่าผิดพลาดเพื่อสาธิต
df['col4'] = ['1', '2', '3', 'four', '5', '6', '7', '8', '9', '10']
print("\nDataFrame ก่อนแปลงค่าผิดพลาด:")
display(df)


In [ ]:

df['col4'] = pd.to_numeric(df['col4'], errors='coerce')
print("\nDataFrame หลังแปลงค่าผิดพลาด:")
display(df)

### 2.3 แสดงตัวอย่างแถวที่มีค่าผิดพลาด (NaN ที่เกิดจากการแปลง)

In [ ]:
print("\nแสดงแถวที่มีค่าผิดพลาดใน 'col4':")
display(df[df['col4'].isna()])

## 3. การตรวจหา Missing Values

Missing values หรือค่าที่หายไป เป็นปัญหาที่พบบ่อยในชุดข้อมูล เราจะมาดูวิธีการตรวจหาค่าเหล่านี้

### 3.1 นับจำนวน missing ต่อคอลัมน์

In [ ]:
print("\nจำนวน missing ต่อคอลัมน์:")
display(df.isnull().sum())

### 3.2 แสดง missing matrix ด้วย `msno.matrix(df)`

Missingno เป็นไลบรารีที่ช่วยให้เห็นภาพรวมของ missing values ได้ง่ายขึ้น `msno.matrix` จะแสดงภาพเป็นเมทริกซ์ที่เห็นตำแหน่งของค่าที่หายไป

In [ ]:
print("\nMissing matrix:")
msno.matrix(df)

### 3.3 แสดง bar chart ของ missing ด้วย `msno.bar(df)`

`msno.bar` จะแสดงเป็นกราฟแท่งที่บอกสัดส่วนของข้อมูลที่มีอยู่ (ไม่ใช่ missing) ในแต่ละคอลัมน์

In [ ]:
print("\nBar chart ของ missing values:")
msno.bar(df)

## 4. กลยุทธ์การจัดการ Missing Values

เมื่อพบ missing values แล้ว เรามีหลายวิธีในการจัดการกับมัน ขึ้นอยู่กับลักษณะของข้อมูลและปัญหาที่เราต้องการแก้ไข

### 4.1 Drop

การลบแถวหรือคอลัมน์ที่มี missing values ทิ้ง

In [ ]:
print("\nDataFrame เดิม:")
display(df)

# สร้างสำเนาเพื่อไม่ให้กระทบ DataFrame เดิม
df_dropped = df.copy()

# ลบแถวที่มีค่า missing ในคอลัมน์ 'col1' หรือ 'col2'
df_dropped.dropna(subset=['col1', 'col2'], inplace=True)
print("\nDataFrame หลัง dropna ใน 'col1' และ 'col2':")
display(df_dropped)

### 4.2 Fill ด้วยสถิติ

การเติมค่า missing ด้วยค่าทางสถิติ เช่น ค่าเฉลี่ย หรือ ค่ามัธยฐาน ของคอลัมน์นั้นๆ

#### ค่าเฉลี่ย: `df[col].fillna(df[col].mean())`

In [ ]:
df_filled_mean = df.copy()
df_filled_mean['col1'].fillna(df_filled_mean['col1'].mean(), inplace=True)
print("\nDataFrame หลัง fillna ด้วยค่าเฉลี่ยใน 'col1':")
display(df_filled_mean)

#### ค่ามัธยฐาน: `df[col].fillna(df[col].median())`

In [ ]:
df_filled_median = df.copy()
df_filled_median['col2'].fillna(df_filled_median['col2'].median(), inplace=True)
print("\nDataFrame หลัง fillna ด้วยค่ามัธยฐานใน 'col2':")
display(df_filled_median)

### 4.3 Fill แบบ Forward/Backward

การเติมค่า missing ด้วยค่าจากแถวก่อนหน้า (ffill) หรือแถวถัดไป (bfill)

#### `df.fillna(method='ffill')` / `df.fillna(method='bfill')`

In [ ]:
df_filled_ffill = df.copy()
df_filled_ffill.fillna(method='ffill', inplace=True)
print("\nDataFrame หลัง fillna ด้วย method='ffill':")
display(df_filled_ffill)



In [ ]:
df_filled_bfill = df.copy()
df_filled_bfill.fillna(method='bfill', inplace=True)
print("\nDataFrame หลัง fillna ด้วย method='bfill':")
display(df_filled_bfill)

## 5. เปรียบเทียบผลก่อน–หลัง Imputation

หลังจากจัดการ missing values แล้ว ควรเปรียบเทียบผลลัพธ์ก่อนและหลัง เพื่อดูว่าการจัดการ missing values ส่งผลต่อการกระจายตัวของข้อมูลอย่างไร

### ใช้ `df.describe()` ก่อนและหลังเติมค่า

In [ ]:
print("\nDescribe ก่อน fillna:")
display(df.describe())

print("\nDescribe หลัง fillna ด้วยค่าเฉลี่ยใน 'col1' และค่ามัธยฐานใน 'col2':")
# ใช้ DataFrame ที่เติมค่าด้วยทั้งสองวิธี
df_combined_fill = df.copy()
df_combined_fill['col1'].fillna(df_combined_fill['col1'].mean(), inplace=True)
df_combined_fill['col2'].fillna(df_combined_fill['col2'].median(), inplace=True)
display(df_combined_fill.describe())

### วาดกราฟเปรียบเทียบ (เช่น histogram)

การเปรียบเทียบด้วยกราฟ Histogram สามารถช่วยให้เห็นการเปลี่ยนแปลงการกระจายตัวของข้อมูลได้ชัดเจน

In [ ]:
import matplotlib.pyplot as plt

print("\nHistogram before and after fillna สำหรับ 'col1':")
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['col1'].dropna(), bins=5, alpha=0.7, label='before fillna')
plt.title('Histogram before fillna (col1)')
plt.xlabel('value')
plt.ylabel('frequency')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(df_combined_fill['col1'], bins=5, alpha=0.7, label='after fillna')
plt.title('Histogram after fillna (col1)')
plt.xlabel('value')
plt.ylabel('frequency')
plt.legend()

plt.tight_layout()
plt.show()


print("\nHistogram before and after fillna สำหรับ 'col2':")
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['col2'].dropna(), bins=5, alpha=0.7, label='before fillna')
plt.title('Histogram before fillna (col2)')
plt.xlabel('value')
plt.ylabel('frequency')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(df_combined_fill['col2'], bins=5, alpha=0.7, label='after fillna')
plt.title('Histogram after fillna (col2)')
plt.xlabel('value')
plt.ylabel('frequency')
plt.legend()

plt.tight_layout()
plt.show()

### สรุป
ความสำคัญของการตรวจสอบ Missing Values และ Data Errors

- **ยืนยันความสมบูรณ์ของข้อมูล**  
  - Missing values และ data errors เป็นสาเหตุหลักที่ทำให้การวิเคราะห์เกิด bias หรือผลลัพธ์ไม่แม่นยำ  
  - การตรวจพบและจัดการตั้งแต่ต้น ป้องกันปัญหาในขั้นตอนถัดไป

- **สร้างความน่าเชื่อถือให้กับสถิติพื้นฐาน**  
  - ค่า mean, median, variance และ distribution จะบิดเบือนได้หากมีค่าที่หายหรือผิดพลาด  
  - ทำให้การสรุปสถิติเบื้องต้น (EDA) มีความถูกต้อง

- **เลือกวิธีจัดการที่เหมาะสมกับลักษณะข้อมูล**  
  - การ drop, fill, forward/backward fill แต่ละวิธีส่งผลต่างกันต่อการวิเคราะห์  
  - การเข้าใจข้อดีข้อจำกัดของแต่ละวิธี ช่วยให้ตัดสินใจได้อย่างรอบคอบ

- **ปูพื้นสำหรับขั้นตอน Modeling ต่อไป**  
  - ข้อมูลที่มีคุณภาพสูง ลดโอกาส overfitting และ error ในการสร้างโมเดล  
  - ช่วยให้โมเดลเรียนรู้ pattern จริง ๆ ของข้อมูล แทนที่จะเรียนรู้จาก noise

- **เพิ่มประสิทธิภาพในการทำงานร่วมกัน**  
  - วิธีจัดการ missing และ errors ที่เป็นมาตรฐาน ทำให้ทีมงานเข้าใจ workflow เดียวกัน  
  - Documentation ใน notebook ช่วยให้ reproducible และตรวจสอบย้อนหลังได้ง่าย